In [1]:
import pandas as pd
import regex as re

In [2]:
# Prepare the occupation level WFH scores

WFH_occ = pd.read_csv(r"..\Data\Source\occupations_workathome.csv")

WFH_occ['occ_code'] = WFH_occ['onetsoccode'].str.strip().str[:7]

WFH_occ = WFH_occ.groupby('occ_code').teleworkable.agg('mean').reset_index()

In [3]:
industry_SOC_xw = {
    'Management occupation':11,
    'Business and financial operations occupation':13,
    'Computer and mathematical occupation':15,
    'Architecture and engineering occupation':17,
    'Life, physical, and social science occupation':19,
    'Community and social service occupation':21,
    'Legal occupation':23,
    'Educational instruction, and library occupation':25,
    'Arts, design, entertainment, sports, and media occupation':27,
    'Healthcare practitioners and technical occupations':29,
    'Health diagnosing and treating practitioners and other technical occupation':29,
    'Health technologists and technician':29,
    'Healthcare support occupation':31,
    'Protective service occupations':33,
    'Firefighting and prevention, and other protective service workers including supervisor':33,
    'Law enforcement workers including supervisor':33,
    'Food preparation and serving related occupation':35,
    'Building and grounds cleaning and maintenance occupation':37,
    'Personal care and service occupation':39,
    'Sales and related occupation':41,
    'Office and administrative support occupation':43,
    'Farming, fishing, and forestry occupation':45,
    'Construction and extraction occupation':47,
    'Installation, maintenance, and repair occupation':49,
    'Production occupation':51,
    'Transportation occupation':53,
    'Material moving occupation':53
}

In [4]:
# Prepare the occupation counts from the census

census = pd.read_csv(r"..\Data\Source\ACS5_2019_S2401_03192021.csv",
                     skiprows=1)

census = census \
    .drop(
        [col for col in census.columns \
         if ((len(col.split("!!")) <= 3) \
         | ("Total" not in col) \
         | ("Estimate" not in col)) \
         & (col != "id")], 
    axis = 1) \
    

census.columns = ['tract'] + [string.split("!!")[-1][:-1] for string in census.columns if len(string.split("!!")) > 3]

census['tract'] = census['tract'].str.strip().str[-11:]

census = census.set_index('tract')

census = census \
    .drop([col for col in census.columns if col not in industry_SOC_xw.keys()],
          axis = 1)

census = pd.melt(census.reset_index(), 
                 id_vars = ['tract'], 
                 value_vars = census.columns)

census['cat'] = census.variable.replace(industry_SOC_xw)

census = census.groupby(['tract', 'cat']).value.agg('sum').reset_index()

In [5]:
# Load the TRACT to ZIP crosswalk file

zip_tract_xw = pd.read_excel(r"..\Data\Source\ZIP_TRACT_032020.xlsx")

zip_tract_xw['ZIP'] = zip_tract_xw['ZIP'].astype(str).str.zfill(5)

zip_tract_xw['TRACT'] = zip_tract_xw['TRACT'].astype(str).str.zfill(11)

In [6]:
# Load the MSA to ZIP crosswalk file

zip_msa_xw = pd.read_csv(r"..\Data\Intermediate\MSA_distance.csv") \
    [['CBSA', 'ZIP']]

zip_msa_xw['ZIP'] = zip_msa_xw['ZIP'].astype(str).str.zfill(5)

In [7]:
# Load the detailed occupation composition per MSA

foo = pd.read_excel(r"..\Data\Source\MSA_M2019_dl.xlsx")

foo['area_title'] = foo['area_title'] \
    .replace({'Miami-Fort Lauderdale-West Palm Beach, FL': "Miami-Fort Lauderdale-Pompano Beach, FL",
              'Atlanta-Sandy Springs-Roswell, GA': "Atlanta-Sandy Springs-Alpharetta, GA",
              'Phoenix-Mesa-Scottsdale, AZ': "Phoenix-Mesa-Chandler, AZ",
              'Boston-Cambridge-Nashua, MA-NH': "Boston-Cambridge-Newton, MA-NH",
              'San Francisco-Oakland-Hayward, CA': "San Francisco-Oakland-Berkeley, CA",
              'San Diego-Carlsbad, CA': "San Diego-Chula Vista-Carlsbad, CA",
              'Sacramento--Roseville--Arden-Arcade, CA': "Sacramento-Roseville-Folsom, CA",
              'Austin-Round Rock, TX': "Austin-Round Rock-Georgetown, TX"})

foo = foo \
    .loc[(foo["jobs_1000"] != '**') & (foo['o_group'] == 'detailed'), ['area_title', 'occ_code', 'jobs_1000']] \
    .merge(WFH_occ, on = 'occ_code', how = 'inner')

foo['cat'] = foo['occ_code'].str.strip().str[:2].astype(int)
foo['total'] = foo.groupby(['area_title', 'cat']).jobs_1000.transform(sum)
foo['w_teleworkable'] = foo['jobs_1000'] * foo['teleworkable'] / foo['total']
foo['w_teleworkable'] = foo['w_teleworkable'].astype(float)
foo = foo \
    .rename(columns = {'area_title': 'CBSA'}) \
    .merge(zip_msa_xw, on = 'CBSA', how = 'inner') \
    .groupby(['ZIP', 'cat']) \
    .w_teleworkable.agg(sum) \
    .reset_index()


In [8]:
# Calculate the tract level WFH score

df = zip_tract_xw \
    [['ZIP', 'TRACT', 'TOT_RATIO']] \
    .rename(columns = {'TRACT': 'tract'}) \
    .merge(census, on = 'tract', how = 'inner') \
    .groupby(['ZIP', 'cat']) \
    .value.agg(sum) \
    .reset_index() \
    .merge(foo, on = ['ZIP', 'cat'], how = 'inner')

df = df[df["value"] > 0]
df['w_teleworkable'] = df['value']*df['w_teleworkable']/df.groupby('ZIP').value.transform('sum')

df = df \
    .groupby('ZIP') \
    .w_teleworkable.agg(sum) \
    .reset_index() \
    .rename(columns = {'w_teleworkable': 'teleworkable'})
df["teleworkable"] = df["teleworkable"].astype(float)

In [9]:
# Save the file
df \
    .rename(columns = {'ZIP': 'zip'}) \
    .to_stata(r"..\Data\Intermediate\WFH_zip.dta", version = 119)

In [10]:
# Convert the MSA-level data from csv to dta

cbsa_map = pd.read_excel(r"..\Data\Source\CBSAs.xls", skiprows = 2)[:-4] \
    .loc[:,['CBSA Code', 'CBSA Title']] \
    .drop_duplicates() \
    .reset_index(drop = True)

df_cbsa = pd.read_csv(r"..\Data\Source\MSA_workfromhome.csv")
df_cbsa['AREA'] = df_cbsa['AREA'].astype(str).replace('71650', '14460')
df_cbsa \
    .merge(cbsa_map, left_on = 'AREA', right_on = 'CBSA Code', how = 'left') \
    .rename(columns = {'CBSA Title': 'cbsa', 'teleworkable_emp': 'teleworkable_cbsa'}) \
    [['cbsa', 'teleworkable_cbsa']] \
    .dropna(subset = ['cbsa']) \
    .to_stata(r"..\Data\Intermediate\WFH_cbsa.dta", version = 119)